# **Cargar  Consolidado**

In [1]:
import pandas as pd

# Ruta del archivo Excel(Cambie esto por la ruta de su consolidado de motores)
ruta_archivo = '/content/PARQUE DE MOTORES - CONSOLIDADO.xlsx'

# Cargar el archivo Excel en un DataFrame
consolidado = pd.read_excel(ruta_archivo, sheet_name='Consolidado') #Cambie el nombre de la hoja por la de su archivo

# **SEGMENTACIÓN DE VOLTAJE**

Visualización los datos de voltaje para tener una idea de como sera la segmentación

In [3]:
voltajes = consolidado['VOLTAJE']
conteos = voltajes.value_counts()
print(conteos)

VOLTAJE
575         733
550         288
4000         30
380          18
4160          9
580           8
480           4
400/690       4
220           3
230           3
230/460       2
13200         2
400/690       1
230/400       1
225/380       1
572           1
585           1
Name: count, dtype: int64


In [4]:
voltajes_exist= list(set(voltajes))
print(voltajes_exist)

[4160, 4000, 480, '230/400', '225/380', 580, 550, 230, '400/690', 585, 575, 13200, '400/690 ', '230/460', 380, 220, 572, nan]


In [5]:
import numpy as np
# Definir segmentos de voltaje
bajovolt1 = [480, 230, '230/460', '225/380','230/400',380, 220,400,'400/690 ','400/690']
bajovolt2 = [550,575,572,585,580]
med_volt = [4160,4000]
alto_volt = [13200]
# Definir los valores que se quieren agregar a la nueva columna para cada segmento
valores = ['BAJO VOLTAJE 1(220-480)', 'BAJO VOLTAJE 2(550-585)', 'MEDIO VOLTAJE(4000/4160)', 'ALTO VOLTAJE(13200)', 'NAN']

def segmentacion_voltaje(consolidado, bajovolt1, bajovolt2,med_volt, alto_volt, valores):
  # Definir las condiciones
  condiciones = [
      consolidado['VOLTAJE'].isin(bajovolt1), # Use isin() to check if each value in the Series is present in the list
      consolidado['VOLTAJE'].isin(bajovolt2),
      consolidado['VOLTAJE'].isin(med_volt),
      consolidado['VOLTAJE'].isin(alto_volt),
      consolidado['VOLTAJE'].isna() # Use isna() to check for NaN values
  ]
      # Añadir una nueva columna basada en las condiciones
  consolidado['SEG_VOLT'] = np.select(condiciones, valores, default='Desconocido')

  return consolidado

In [6]:
consolidado = segmentacion_voltaje(consolidado, bajovolt1, bajovolt2,med_volt, alto_volt, valores)

In [15]:
conteos = consolidado['SEG_VOLT'].value_counts()
print(conteos)

SEG_VOLT
BAJO VOLTAJE 2(550-585)     1031
NAN                           67
MEDIO VOLTAJE(4000/4160)      39
BAJO VOLTAJE 1(220-480)       37
ALTO VOLTAJE(13200)            2
Name: count, dtype: int64


# **CALCULO DE NUMERO DE POLOS Y FRECUENCIA (con la columna RPM)**

In [10]:
def determinar_polos_frecuencia(rpm):
    rangos_60hz = {
        3600: (2, 60),
        1800: (4, 60),
        1200: (6, 60),
        900: (8, 60),
        720: (10, 60),
        600: (12, 60),
        514: (14, 60),
        450: (16, 60),
        400: (18, 60),
        360: (20, 60),
        327: (22, 60),
        300: (24, 60)
    }
    rangos_50hz = {
        3000: (2, 50),
        1500: (4, 50),
        1000: (6, 50),
        750: (8, 50),
        600: (10, 50),
        500: (12, 50),
        428: (14, 50),
        375: (16, 50),
        333: (18, 50),
        300: (20, 50)
    }

    def buscar_valor_mas_cercano(rpm, rangos):
        # Buscar el valor más cercano superior
        valor_mas_cercano = None
        for valor_rpm in sorted(rangos.keys()):
            if rpm <= valor_rpm:
                valor_mas_cercano = valor_rpm
                break
        if valor_mas_cercano is not None:
            return rangos[valor_mas_cercano]
        return (None, None)

    # Buscar el valor más cercano para 60 Hz
    resultado_60hz = buscar_valor_mas_cercano(rpm, rangos_60hz)

    # Buscar el valor más cercano para 50 Hz
    resultado_50hz = buscar_valor_mas_cercano(rpm, rangos_50hz)

    # Elegir el resultado que tenga la menor diferencia de RPM
    if resultado_50hz[0] is not None and resultado_60hz[0] is not None:
        diff_50hz = abs(rpm - next(val for val in sorted(rangos_50hz.keys()) if rpm <= val))
        diff_60hz = abs(rpm - next(val for val in sorted(rangos_60hz.keys()) if rpm <= val))

        if diff_50hz < diff_60hz:
            return resultado_50hz
        else:
            return resultado_60hz
    elif resultado_50hz[0] is not None:
        return resultado_50hz
    elif resultado_60hz[0] is not None:
        return resultado_60hz
    else:
        return (None, None)

In [11]:
# Aplicar la función para determinar polos y frecuencia
consolidado['Polos'], consolidado['Frecuencia'] = zip(*consolidado['R.P.M'].apply(determinar_polos_frecuencia))

In [13]:
conteos = consolidado['Frecuencia'].value_counts()
print(conteos)

Frecuencia
50.0    1075
60.0      27
Name: count, dtype: int64


In [14]:
conteos = consolidado['Polos'].value_counts()
print(conteos)

Polos
4.0     839
6.0     141
8.0      62
2.0      33
24.0     14
16.0      7
14.0      4
12.0      2
Name: count, dtype: int64


In [17]:
# Descargar el DataFrame como un archivo Excel
consolidado.to_excel('consolidado_motores1.xlsx', index=False)
 #puede descargarlo con el nombre que quiera, si usa colab este se guardara en archivos y lo tiene que descargar manual